# CNN Fashion MNIST


<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/Lorenzo-Giardi/tf-keras/blob/master/5_CNN/cnn_fashion_mnist.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

In [2]:
import keras
import tensorflow as tf
import numpy as np
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
from keras.losses import mean_squared_error
from keras.models import Sequential
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout

np.random.seed(42)
tf.random.set_seed(42)

In [3]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

X_mean = X_train.mean(axis=0, keepdims=True)
X_std = X_train.std(axis=0, keepdims=True) + 1e-7
X_train = (X_train - X_mean) / X_std
X_valid = (X_valid - X_mean) / X_std
X_test = (X_test - X_mean) / X_std

X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [4]:
labels = ['T-shirt', 'Troursers', 'Pullover', 'Dress', 'Coat', 'Sandal',
              'Shirt', 'Sneaker', 'Bag', 'Ankle_boot']

### Network

In [5]:
model = Sequential([
    Conv2D(64, 7, activation='relu', padding = 'same', input_shape = [28, 28, 1]),
    MaxPooling2D(2),
    Conv2D(128, 3, activation='relu', padding = 'same'),
    Conv2D(128, 3, activation='relu', padding = 'same'),
    MaxPooling2D(2),
    Conv2D(256, 3, activation='relu', padding = 'same'),
    Conv2D(256, 3, activation='relu', padding = 'same'),
    MaxPooling2D(2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

In [6]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 64)        3200      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 256)         295168    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 256)        

In [7]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs = 10, validation_data = (X_valid, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 360s 7ms/step - loss: 0.5866 - accuracy: 0.7890 - val_loss: 0.3866 - val_accuracy: 0.8544
Epoch 2/10
48640/55000 [=========================>....] - ETA: 41s - loss: 0.3890 - accuracy: 0.8604